Projekt uporablja knjižnice:
1. **tensorflow** za konstrukcijo nevronske mreže in strojno učenje
2. **pillow** za prikazovanje slik
3. **numpy** za pretvorbo podatkov v tip s katerim deluje tensorflow

In [9]:
import tensorflow as tf #The project was made with version 2.1.0
from PIL import Image
import numpy as np

Najprej potrebujemo podatke za učenje modela, za kar bomo uporabili nabor podatkov ročno napisanih števk MNIST s strani http://yann.lecun.com/exdb/mnist/ . To je pogosto uporabljena knjižnica pri projektih z nevronskimi mrežami tako da tensorflow že vsebuje podatke te knjižnice v tf.keras.datasets.mnist

x_train in y_train vsebuje podatke in odgovore, ki bodo uporabljeni pri učenju, x_test in y_test pa podatke ter odgovore, ki bodo uporabljeni pri preverjanju uspešnosti mreže.

In [10]:
#Select mnist dataset
mnist = tf.keras.datasets.mnist

#Load data
(x_train, y_train), (x_test, y_test) = mnist.load_data()

#And convert to float
x_train, x_test = x_train / 255.0, x_test / 255.0

Sledi kreacija nevronske mreže. tf.keras.models.Sequential je standardna feedforward nevronska mreža s štirimi plastmi.
1. Prva plast je Flatten, ki sprejme 2D list oblike 28x28 (sliko števke).
2. Druga skrita plast ima 128 nevronov z aktivacijsko funkcijo relu.
3. Med drugo in zadnjo plastjo je še plast Dropout, ki med učenjem naključno izkljaplja nevrone kar izboljša povezavo med sosednjimi nevroni in ustvari boljši rezultat učenja.
4. Zadnja plast ima 10 nevronov, vsak izmed katerih predstavlja eno števko od 0 do 9.

Vse uteži in biasi nevronov so na začetku naključno izbrani.

In [11]:
#Create neural network
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10)
])

Pred učenjem potrebuje mreža še funkcijo izgube, ki primerja odgovor mreže s pravilnim odgovorom ter izračuna velikost napake nevronske mreže. Napaka je 0, če mreža napove pravilen odgovor.  

In [12]:
#Initialize loss function
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

Pri compilanju modela izberemo optimizer adam, funkcijo izgube in kaj poiskušamo doseči pri učenju. V tem primeru želimo optimizirati natančnost nevronske mreže.

In [13]:
#Compile model for training
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

Nevronsko mrežo bomo na učnih podatkih učili 5 epoh, kar pomeni da bomo skozi mrežo 5-krat poslali učne podatke in prilagodili uteži in povezave med nevroni da bomo izboljšali natančnost mreže. Ob koncu vsake epohe izpišemo izgubo mreže in njeno natančnost. Čez epohe se izguba mreže manjša in njena natančnost veča. Opazno je, da za natančnost cca 90% potrebujemo samo eno epoho, da dosežemo natančnost 97-98% pa potrebujemo še 4 nadaljnje epohe.

In [14]:
#Train model
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 17s 284us/sample - loss: 0.2964 - accuracy: 0.9148
Epoch 2/5
60000/60000 [==============================] - 17s 284us/sample - loss: 0.1401 - accuracy: 0.9582
Epoch 3/5
60000/60000 [==============================] - 18s 299us/sample - loss: 0.1061 - accuracy: 0.9675
Epoch 4/5
60000/60000 [==============================] - 19s 319us/sample - loss: 0.0864 - accuracy: 0.9733
Epoch 5/5
60000/60000 [==============================] - 19s 317us/sample - loss: 0.0751 - accuracy: 0.9764


Za natančno evaluacijo modela ga je potrebno preiskusiti na podatkih, ki niso bili vključeni v učne podatke. S tem preverimo natančnost in se prepričamo, da model deluje na novih podatkih in ni overfitan.

In [15]:
#Evaluate model on new data
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 2s - loss: 0.0745 - accuracy: 0.9765


[0.0745175982091343, 0.9765]

Preden začnemo uporabljati model ga spremenimo v probability_model tako, da mu dodamo še eno plast, ki spremeni dejanski output modela v verjetnost, ki je bolj človeško berljiva kot dejanski output.

In [17]:
#Add probability function to create new model with human output
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

Spodnja koda izbere naključni primer iz testnih podatkov, ga prikaže kot sliko in izpiše napoved nevronske mreže ter pravilni odgovor.

In [38]:
def print_output(alist):
    m = max(alist)
    print("digit : probability")
    for i in range(10):
        if alist[i] == m:
            print(f"--> {i} : {alist[i]}")
        else:
            print(f"    {i} : {alist[i]}")

In [39]:
#Show random image from test data

(_, _), (x_test, y_test) = mnist.load_data() #x_test must be int so pil works

i = np.random.randint(0, len(x_test))

Image.fromarray(x_test[i]).show() #x_test must be int so this line works

a = np.ndarray(shape=(1,28,28))
a[0] = x_test[i] / 255 #Convert back to 

print_output(probability_model(a).numpy()[0])
print(f"Correct answer = {y_test[i]}")

digit : probability
    0 : 8.556535924242326e-09
    1 : 4.525457508730568e-10
    2 : 1.036860153291741e-09
    3 : 4.924270797346253e-06
    4 : 1.944005452969577e-05
    5 : 3.927866146113956e-06
    6 : 1.0058408966839849e-10
    7 : 7.40578207114595e-06
    8 : 1.3655082511832006e-05
--> 9 : 0.9999505281448364
Correct answer = 9


Spodnja koda prebere sliko digit.png, ki mora biti 28x28 digit kjer je ozadje črno in števka narisana z belo barvo ter izpiše napoved nevronske mreže.

In [41]:
#Test on self made image

im = Image.open("digit.png").convert("L")

a = np.ndarray(shape=(1,28,28))
a[0] = np.asarray(im) / 255

print_output(probability_model(a).numpy()[0])

digit : probability
    0 : 0.000571421580389142
    1 : 0.001965818228200078
    2 : 0.0003358131507411599
    3 : 0.021000295877456665
    4 : 0.0022862672340124846
--> 5 : 0.8722296953201294
    6 : 0.00016511617286596447
    7 : 0.01023405883461237
    8 : 0.08958566933870316
    9 : 0.001625954289920628
